In [3]:
import pickle

file_path = '../my_app/data/ingr_map.pkl'

with open(file_path, 'rb') as file:
    ingr_map = pickle.load(file)

print(type(ingr_map))
print(ingr_map)



<class 'pandas.core.frame.DataFrame'>
                                                raw_ingr  raw_words   
0      medium heads bibb or red leaf lettuce, washed,...         13  \
1                  mixed baby lettuces and spring greens          6   
2                                   romaine lettuce leaf          3   
3                                   iceberg lettuce leaf          3   
4                                    red romaine lettuce          3   
...                                                  ...        ...   
11654                                           soybeans          1   
11655                                              goose          1   
11656                                             ajwain          1   
11657                                           brinjals          1   
11658                                              khoya          1   

                                               processed  len_proc replaced   
0      medium heads bibb or re

In [4]:
# Basic information
print(ingr_map.info())

# Display first few rows
print(ingr_map.head())

# Summary statistics
print(ingr_map.describe())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11659 entries, 0 to 11658
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   raw_ingr   11659 non-null  object
 1   raw_words  11659 non-null  int64 
 2   processed  11659 non-null  object
 3   len_proc   11659 non-null  int64 
 4   replaced   11659 non-null  object
 5   count      11659 non-null  int64 
 6   id         11659 non-null  int16 
dtypes: int16(1), int64(3), object(3)
memory usage: 660.4+ KB
None
                                            raw_ingr  raw_words   
0  medium heads bibb or red leaf lettuce, washed,...         13  \
1              mixed baby lettuces and spring greens          6   
2                               romaine lettuce leaf          3   
3                               iceberg lettuce leaf          3   
4                                red romaine lettuce          3   

                                           processed  len_proc replaced

In [14]:
import numpy as np

# Convert to a NumPy array and back to DataFrame
ingr_map = pd.DataFrame(np.array(ingr_map), columns=ingr_map.columns)

# Save the cleaned DataFrame as CSV
ingr_map.to_csv('ingr_map_clean.csv', index=False)

NameError: name 'pd' is not defined

In [ ]:
# Save as CSV
# Reset the index to avoid any issues with the DataFrame's structure
ingr_map = ingr_map.reset_index(drop=True)

ingr_map.to_csv('../my_app/data/ingr_map.csv', index=False)

# Save as Parquet (for large datasets)
ingr_map.to_parquet('../my_app/data/ingr_map.parquet', index=False)


AttributeError: 'Index' object has no attribute '_format_native_types'

In [1]:
import pandas as pd
import ast  # Use to safely evaluate stringified lists

def split_nutrition_column(df):
    """
    Splits the nutrition column into individual columns.
    Args:
        df (pd.DataFrame): The recipes DataFrame containing a 'nutrition' column.
    Returns:
        pd.DataFrame: DataFrame with additional nutrition columns.
    """
    # Define column names for nutrition components
    nutrition_labels = [
        "calories", 
        "total_fat_pdv", 
        "sugar_pdv", 
        "sodium_pdv", 
        "protein_pdv", 
        "saturated_fat_pdv", 
        "carbohydrates_pdv"
    ]
    
    # Parse and expand the nutrition column
    def parse_nutrition(nutrition):
        try:
            # Convert stringified list to actual list
            return ast.literal_eval(nutrition)
        except (ValueError, SyntaxError):
            # Return NaN if parsing fails
            return [None] * len(nutrition_labels)
    
    # Apply parsing and create new columns
    nutrition_expanded = df['nutrition'].apply(parse_nutrition).tolist()
    nutrition_df = pd.DataFrame(nutrition_expanded, columns=nutrition_labels)
    
    # Concatenate the new columns with the original DataFrame
    df = pd.concat([df, nutrition_df], axis=1)
    return df

# Load the RAW_recipes.csv file
raw_recipes = pd.read_csv('../my_app/data/RAW_recipes.csv')

# Split the nutrition column into separate columns
raw_recipes = split_nutrition_column(raw_recipes)

# Save the updated DataFrame to a new CSV file if needed
raw_recipes.to_csv('../my_app/data/RAW_recipes.csv', index=False)

# Display the first few rows of the updated DataFrame
print(raw_recipes.head())


                                         name      id  minutes  \
0  arriba   baked winter squash mexican style  137739       55   
1            a bit different  breakfast pizza   31490       30   
2                   all in the kitchen  chili  112140      130   
3                          alouette  potatoes   59389       45   
4          amish  tomato ketchup  for canning   44061      190   

   contributor_id   submitted  \
0           47892  2005-09-16   
1           26278  2002-06-17   
2          196586  2005-02-25   
3           68585  2003-04-14   
4           41706  2002-10-25   

                                                tags  \
0  ['60-minutes-or-less', 'time-to-make', 'course...   
1  ['30-minutes-or-less', 'time-to-make', 'course...   
2  ['time-to-make', 'course', 'preparation', 'mai...   
3  ['60-minutes-or-less', 'time-to-make', 'course...   
4  ['weeknight', 'time-to-make', 'course', 'main-...   

                                    nutrition  n_steps  \
0       [

In [2]:
raw_recipes.describe()

,id,minutes,contributor_id,n_steps,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000,231637.000000,231637.00000,231637.000000,231637.000000,231637.00000,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153,473.942425,36.08070,84.296865,30.147485,34.68186,45.589150,15.560403
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796,1189.711374,77.79884,800.080897,131.961589,58.47248,98.235758,81.824560
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000,174.400000,8.00000,9.000000,5.000000,7.00000,7.000000,4.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000,313.400000,20.00000,25.000000,14.000000,18.00000,23.000000,9.000000
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000,519.700000,41.00000,68.000000,33.000000,51.00000,52.000000,16.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000,434360.200000,17183.00000,362729.000000,29338.000000,6552.00000,10395.000000,36098.000000


In [5]:
import pandas as pd

# Load the dataset
file_path = "../my_app/data/RAW_recipes.csv"  # Update with your actual dataset path
df = pd.read_csv(file_path)

# Conversion factors for 100% Daily Value
conversion_factors = {
    'total_fat_pdv': 78,          # Total fat in grams
    'saturated_fat_pdv': 20,      # Saturated fat in grams
    'protein_pdv': 50,            # Protein in grams
    'carbohydrates_pdv': 275,     # Carbohydrates in grams
    'sodium_pdv': 2300            # Sodium in mg
}

# Convert PDV columns to grams/mg
for column, factor in conversion_factors.items():
    new_column = column.replace('_pdv', '_grams')
    df[new_column] = (df[column] * factor) / 100

# Save the updated dataset
output_path = "../my_app/data/RAW_recipes.csv"
df.to_csv(output_path, index=False)
print(f"Updated dataset saved to {output_path}")


Updated dataset saved to ../my_app/data/RAW_recipes.csv
